# Data Mart Smartphones — Tổng quan quy trình

Notebook này thực hiện các bước để làm sạch dữ liệu smartphone, chuẩn hoá phục vụ trực quan hoá, và xây dựng data mart theo mô hình sao (star schema) gồm các bảng dimension và fact.

## Nội dung từng bước

1) Cài đặt/thư viện
- Cài pandas để xử lý dữ liệu.

2) Khảo sát dữ liệu thô
- Đọc `smartphones_cleaned_v6.csv`.
- Thống kê số lượng giá trị thiếu theo cột.

3) Đổi tên cột Anh → Việt (chuẩn hoá schema nguồn)
- Ánh xạ tên cột sang tiếng Việt, bổ sung cả `Độ phân giải dọc` để tính PPI.
- Lưu thành `smartphones_vietnamese.csv`.

4) Xử lý giá trị thiếu (imputation)
- Điền thiếu cho: Đánh giá, Hãng CPU, Số nhân CPU, Tốc độ CPU theo mean theo hãng, Dung lượng pin theo median, Sạc nhanh ("không hỗ trợ"), Số camera trước, Hệ điều hành (Android), Camera trước (median), Hỗ trợ thẻ nhớ tối đa (0).
- Ghi đè lại `smartphones_vietnamese.csv`.

5) Chuẩn hoá cho trực quan hoá (`smartphones_vietnamese_viz.csv`)
- Làm sạch text nhẹ cho các trường danh mục (thương hiệu, mẫu máy, hãng CPU, hệ điều hành).
- Ép kiểu số cho các trường số (giá, đánh giá, CPU, pin, RAM/ROM, màn hình, tần số, camera, độ phân giải...).
- Sạc nhanh: tách thành 2 cột
  - `Có_sạc_nhanh_(0_1)` (Int64: 0/1/NA)
  - `Công_suất_sạc_nhanh_(W)` (số Watt; 0 nếu không hỗ trợ)
- Hỗ trợ thẻ nhớ: tạo `Có_hỗ_trợ_thẻ_nhớ_(0_1)` từ `Hỗ trợ thẻ nhớ tối đa (GB)`.
- Tính PPI từ `Độ phân giải ngang`, `Độ phân giải dọc`, `Kích thước màn hình (inch)`.
- Tạo nhóm: `Nhóm giá (triệu)`, `Nhóm RAM`, `Nhóm ROM`, `Nhóm màn hình` (loại bỏ dấu ") và thêm `log10_giá`.
- Lưu file `smartphones_vietnamese_viz.csv`.

6) Xây dựng Data Mart (Star Schema) và xuất CSV
- Tạo khóa thay thế ổn định hơn bằng cách sort trước khi gán `smartphone_sk`.
- Dimension:
  - `dim_thuong_hieu`: Thương hiệu + `ma_thuong_hieu`.
  - `dim_cpu`: Hãng CPU, Số nhân, Tốc độ + `ma_cpu`.
  - `dim_man_hinh`: Kích thước, Tần số, Độ phân giải ngang/dọc, `ppi` + `ma_manhinh`.
  - `dim_bo_nho`: RAM, ROM, Hỗ trợ thẻ nhớ tối đa, Có_hỗ_trợ_thẻ_nhớ_(0_1) + `ma_bonho`.
  - `dim_camera`: Camera sau/trước, Số camera sau/trước + `ma_camera`.
  - `dim_os`: Hệ điều hành + `ma_os`.
- Fact `fact_smartphone` (liên kết các dimension) gồm: `smartphone_sk`, các mã dimension, chỉ số giá/đánh giá, nhóm phân khúc, và nếu có thì thêm `Có_sạc_nhanh_(0_1)`, `Công_suất_sạc_nhanh_(W)`.
- Xuất các file CSV vào thư mục `output_dim_fact/`.

## Sản phẩm đầu ra
- `smartphones_vietnamese.csv`: dữ liệu nguồn đã Việt hoá cột + điền thiếu.
- `smartphones_vietnamese_viz.csv`: dữ liệu đã chuẩn hoá/phát sinh biến (PPI, nhóm, sạc nhanh tách cột).
- Thư mục `output_dim_fact/` gồm:
  - `dim_thuong_hieu.csv`, `dim_cpu.csv`, `dim_man_hinh.csv`, `dim_bo_nho.csv`, `dim_camera.csv`, `dim_os.csv`
  - `fact_smartphone.csv`

## Ghi chú
- Khóa ổn định: Đã sort trước khi gán ID; nếu cần ổn định tuyệt đối giữa nhiều lần chạy và thay đổi dữ liệu, có thể dùng băm (hash) thuộc tính.
- Dimension CPU có thể tách thêm dim hãng CPU riêng nếu muốn giảm cardinality.

## Chi tiết các bước xử lý

Dưới đây là mô tả cụ thể hơn theo từng cell trong notebook.

### Bước 2 — Khảo sát dữ liệu thô (Cell 2)
- Input: `smartphones_cleaned_v6.csv`
- Thao tác: đọc file, tính tổng số giá trị thiếu theo cột (`isnull().sum()`), lọc các cột có thiếu (`> 0`).
- Output: in ra các cột có giá trị thiếu để xác định chiến lược điền thiếu ở bước sau.
- Lưu ý: không thay đổi dữ liệu ở bước này, chỉ quan sát.

### Bước 3 — Đổi tên cột Anh → Việt (Cell 3)
- Input: `smartphones_cleaned_v6.csv`
- Thao tác: `rename(columns=col_map)` theo từ điển ánh xạ, bao gồm cả:
  - `resolution_width` → `Độ phân giải ngang`
  - `resolution_height` → `Độ phân giải dọc` (cần để tính PPI)
- Output: `smartphones_vietnamese.csv` (các cột đã chuẩn hoá tên tiếng Việt).
- Lưu ý: không thay đổi giá trị, chỉ đổi tên cột.

### Bước 4 — Xử lý giá trị thiếu (Cell 4)
- Input: `smartphones_vietnamese.csv`
- Thao tác điền thiếu theo từng trường:
  - `Đánh giá`: điền chuỗi "không có đánh giá" nếu thiếu (giúp hiển thị thân thiện trước khi ép kiểu số ở bước 5).
  - `Hãng CPU`: điền mode (giá trị phổ biến nhất).
  - `Số nhân CPU`: điền mode.
  - `Tốc độ CPU (GHz)`: điền theo mean nội nhóm theo `Hãng CPU`.
  - `Dung lượng pin (mAh)`: điền median (giảm ảnh hưởng ngoại lệ).
  - `Sạc nhanh`: điền "không hỗ trợ" nếu thiếu (sẽ tách cột ở bước 5).
  - `Số camera trước`: điền 1 nếu thiếu.
  - `Hệ điều hành`: điền "Android" nếu thiếu.
  - `Camera trước (MP)`: điền median.
  - `Hỗ trợ thẻ nhớ tối đa (GB)`: điền 0.
- Output: ghi đè `smartphones_vietnamese.csv`.
- Lưu ý: điền thiếu theo nguyên tắc đơn giản để giữ lại tối đa bản ghi; có thể tinh chỉnh theo nghiệp vụ.

### Bước 5 — Chuẩn hoá cho trực quan hoá (Cell 6)
- Input: `smartphones_vietnamese.csv`
- Thao tác chính:
  1) Làm sạch text nhẹ (chuẩn hoá chữ cái/dấu cách) cho các trường danh mục: `Thương hiệu`, `Mẫu máy`, `Hãng CPU`, `Hệ điều hành`.
  2) Ép kiểu số an toàn (`to_numeric(errors='coerce')`) cho các trường số (giá, CPU, pin, RAM, ROM, màn hình, tần số, camera, độ phân giải...).
  3) Tách trường "Sạc nhanh" thành 2 cột:
     - `Có_sạc_nhanh_(0_1)`: 0/1/NA theo nội dung chuỗi; nhận diện "không hỗ trợ", "no", "false" → 0; có số watt hoặc "có/yes/true" → 1.
     - `Công_suất_sạc_nhanh_(W)`: số watt trích từ chuỗi bằng regex (`([0-9]+\.?[0-9]*)`), gán 0 nếu không hỗ trợ.
  4) Hỗ trợ thẻ nhớ: tạo `Có_hỗ_trợ_thẻ_nhớ_(0_1)` = 1 nếu `Hỗ trợ thẻ nhớ tối đa (GB)` > 0, ngược lại 0.
  5) Tính PPI: công thức `sqrt(width^2 + height^2) / screen_inches` khi đủ dữ liệu và `screen_inches > 0`.
  6) Tạo biến nhóm (binning):
     - `Nhóm giá (triệu)`: <5M, 5–10M, 10–20M, 20–30M, >30M; đồng thời tạo `log10_giá` cho phân tích phân phối.
     - `Nhóm RAM`: ≤4GB, 6–8GB, 12GB, >12GB.
     - `Nhóm ROM`: ≤64, 128, 256, 512–1TB, >1TB.
     - `Nhóm màn hình`: <6, 6–6.7, >6.7 (loại bỏ dấu ").
- Output: `smartphones_vietnamese_viz.csv` (sẵn sàng cho BI/viz).
- Lưu ý: mọi ép kiểu lỗi đều trở thành NA; các phép tính (PPI/log) chỉ thực hiện khi dữ liệu hợp lệ.

### Bước 6 — Xây dựng Data Mart (Star Schema) (Cell 7)
- Input: `smartphones_vietnamese_viz.csv`
- Ổn định khoá thay thế:
  - Sort theo `Thương hiệu`, `Mẫu máy`, `Giá (VNĐ)` (nếu có) trước khi gán `smartphone_sk` để giảm dao động ID qua các lần chạy.
- Tạo các bảng dimension (sau khi `drop_duplicates()` và `sort_values()` để khoá ổn định):
  - `dim_thuong_hieu(Thương hiệu, ma_thuong_hieu)`
  - `dim_cpu(Hãng CPU, Số nhân CPU, Tốc độ CPU (GHz), ma_cpu)`
  - `dim_man_hinh(Kích thước màn hình (inch), Tần số quét (Hz), Độ phân giải ngang, Độ phân giải dọc, ppi, ma_manhinh)`
  - `dim_bo_nho(Dung lượng RAM (GB), Bộ nhớ trong (GB), Hỗ trợ thẻ nhớ tối đa (GB), Có_hỗ_trợ_thẻ_nhớ_(0_1), ma_bonho)`
  - `dim_camera(Camera sau (MP), Camera trước (MP), Số camera sau, Số camera trước, ma_camera)`
  - `dim_os(Hệ điều hành, ma_os)`
- Tạo bảng fact `fact_smartphone` bằng cách merge theo khoá tự nhiên tương ứng, chọn các cột:
  - Khóa: `smartphone_sk`, `ma_thuong_hieu`, `ma_cpu`, `ma_manhinh`, `ma_bonho`, `ma_camera`, `ma_os`.
  - Chỉ số: `Giá (VNĐ)`, `Đánh giá`.
  - Thuộc tính phân khúc: `Nhóm giá (triệu)`, `Nhóm RAM`, `Nhóm ROM`, `Nhóm màn hình`.
  - Sạc nhanh: ưu tiên dùng `Có_sạc_nhanh_(0_1)`, `Công_suất_sạc_nhanh_(W)` nếu có; nếu không, dùng `Sạc nhanh` gốc.
- Output: CSV tại `output_dim_fact/`: tất cả dimension + `fact_smartphone.csv`.

### Edge cases & Lưu ý chất lượng
- PPI: cần đủ `Độ phân giải ngang`, `Độ phân giải dọc`, `Kích thước màn hình (inch)` > 0; ngược lại để NA.
- Ép kiểu số: giá trị không chuyển được sẽ thành NA; các phép toán có kiểm tra NA để tránh lỗi.
- Sạc nhanh: regex watt có thể bỏ sót định dạng lạ (ví dụ "67W max(USB-PD)"); có thể mở rộng mẫu regex khi cần.
- Khoá ổn định: dùng `sort_values(..., kind='stable')`; để ổn định tuyệt đối qua nhiều lần thay đổi dữ liệu, có thể thay bằng khoá băm (hash) thuộc tính.
- Merge dimension: chỉ merge trên các cột hiện diện; tránh lỗi khi một số cột vắng mặt trong bộ dữ liệu thực tế.
- Encoding: xuất CSV `utf-8-sig` để tương thích Excel/BI tiếng Việt.

### Quy trình chạy lại
- Khi nguồn thay đổi, chạy tuần tự từ Cell 2 → 7 để làm mới toàn bộ đầu ra.
- Nếu chỉ chỉnh logic nhóm/chuẩn hoá, chạy lại Cell 5 → 7 là đủ; nếu đổi mapping cột, chạy lại từ Cell 3.

In [42]:
import pandas as pd
df = pd.read_csv("smartphones_cleaned_v6.csv")
missing_values = df.isnull().sum()
missing_columns = missing_values[missing_values > 0]
print("Các cột có giá trị thiếu:")
print(missing_columns)


Các cột có giá trị thiếu:
rating                  101
processor_brand          20
num_cores                 6
processor_speed          42
battery_capacity         11
fast_charging           211
num_front_cameras         4
os                       14
primary_camera_front      5
extended_upto           480
dtype: int64


In [43]:
# Tạo từ điển ánh xạ cột tiếng Anh → tiếng Việt
col_map = {
    "brand_name": "Thương hiệu",
    "model": "Mẫu máy",
    "os": "Hệ điều hành",
    "battery_capacity": "Dung lượng pin (mAh)",
    "ram_capacity": "Dung lượng RAM (GB)",
    "internal_memory": "Bộ nhớ trong (GB)",
    "extended_upto": "Hỗ trợ thẻ nhớ tối đa (GB)",
    "extended_memory_available": "Có hỗ trợ thẻ nhớ",
    "screen_size": "Kích thước màn hình (inch)",
    "refresh_rate": "Tần số quét (Hz)",
    "resolution_width": "Độ phân giải ngang",
    "resolution_height": "Độ phân giải dọc",  # thêm để tính PPI
    "primary_camera_rear": "Camera sau (MP)",
    "num_rear_cameras": "Số camera sau",
    "primary_camera_front": "Camera trước (MP)",
    "num_front_cameras": "Số camera trước",
    "processor_brand": "Hãng CPU",
    "processor_speed": "Tốc độ CPU (GHz)",
    "num_cores": "Số nhân CPU",
    "fast_charging": "Sạc nhanh",
    "rating": "Đánh giá",
    "price": "Giá (VNĐ)"
}

# Đổi tên cột
df_renamed = df.rename(columns=col_map)

# Lưu lại file mới
df_renamed.to_csv("smartphones_vietnamese.csv", index=False, encoding="utf-8-sig")

print("Đã dịch tên cột và lưu vào smartphones_vietnamese.csv")

Đã dịch tên cột và lưu vào smartphones_vietnamese.csv


In [44]:
# xử lý giá trị thiếu

df = pd.read_csv("smartphones_vietnamese.csv")
# nếu không có đánh giá thì điền chưa có 
df['Đánh giá'] = df['Đánh giá'].fillna('không có đánh giá')
# nếu không có hãng cpu thì điền mode
df['Hãng CPU'] = df['Hãng CPU'].fillna(df['Hãng CPU'].mode()[0])
# điền giá trị cho số nhân cpu bằng mode
df['Số nhân CPU'] = df['Số nhân CPU'].fillna(df['Số nhân CPU'].mode()[0])
# điền tốc độ CPU bằng mean của hãng cpu
df['Tốc độ CPU (GHz)'] = df.groupby('Hãng CPU')['Tốc độ CPU (GHz)'].transform(lambda x: x.fillna(x.mean()))
# điền dung lượng pin bằng median
df['Dung lượng pin (mAh)'] = df['Dung lượng pin (mAh)'].fillna(df['Dung lượng pin (mAh)'].median())
# điền sạc nhanh bằng không hỗ trợ
df['Sạc nhanh'] = df['Sạc nhanh'].fillna('không hỗ trợ')
# điền số cam trước =1
df['Số camera trước'] = df['Số camera trước'].fillna(1)
# điền hệ điều hành là android
df['Hệ điều hành'] = df['Hệ điều hành'].fillna('Android')
# điền cam trước bằng median
df['Camera trước (MP)'] = df['Camera trước (MP)'].fillna(df['Camera trước (MP)'].median())
# điền hỗ trợ bố nhớ tối đa bằng 0
df['Hỗ trợ thẻ nhớ tối đa (GB)'] = df['Hỗ trợ thẻ nhớ tối đa (GB)'].fillna(0)

# in lại kết quả
missing_values = df.isnull().sum()
missing_columns = missing_values[missing_values > 0]
print("Các cột có giá trị thiếu:")
print(missing_columns)
# in dữ liệu
print(df)
# lưu lại trên file smartphones_vietnamese
df.to_csv('smartphones_vietnamese.csv', index=False)

Các cột có giá trị thiếu:
Series([], dtype: int64)
    Thương hiệu                          Mẫu máy  Giá (VNĐ) Đánh giá  has_5g  \
0       oneplus                    OnePlus 11 5G      54999     89.0    True   
1       oneplus        OnePlus Nord CE 2 Lite 5G      19989     81.0    True   
2       samsung            Samsung Galaxy A14 5G      16499     75.0    True   
3      motorola             Motorola Moto G62 5G      14999     81.0    True   
4        realme               Realme 10 Pro Plus      24999     82.0    True   
..          ...                              ...        ...      ...     ...   
975    motorola       Motorola Moto Edge S30 Pro      34990     83.0    True   
976       honor                      Honor X8 5G      14990     75.0    True   
977        poco  POCO X4 GT 5G (8GB RAM + 256GB)      28990     85.0    True   
978    motorola             Motorola Moto G91 5G      19990     80.0    True   
979     samsung           Samsung Galaxy M52s 5G      24990     74.0 

In [47]:
import pandas as pd
import numpy as np
import re

# ========= ĐỌC FILE =========
df = pd.read_csv('smartphones_vietnamese.csv')

# Từ điển cột tiếng Việt
VN = {
    "brand": "Thương hiệu",
    "model": "Mẫu máy",
    "os": "Hệ điều hành",
    "battery": "Dung lượng pin (mAh)",
    "ram": "Dung lượng RAM (GB)",
    "rom": "Bộ nhớ trong (GB)",
    "extended_upto": "Hỗ trợ thẻ nhớ tối đa (GB)",
    "screen": "Kích thước màn hình (inch)",
    "refresh": "Tần số quét (Hz)",
    "res_w": "Độ phân giải ngang",
    "res_h": "Độ phân giải dọc",
    "rear_cam": "Camera sau (MP)",
    "rear_count": "Số camera sau",
    "front_cam": "Camera trước (MP)",
    "front_count": "Số camera trước",
    "cpu_brand": "Hãng CPU",
    "cpu_speed": "Tốc độ CPU (GHz)",
    "cores": "Số nhân CPU",
    "fast": "Sạc nhanh",
    "rating": "Đánh giá",
    "price": "Giá (VNĐ)",
}

def has(key):
    return VN[key] in df.columns

# ========= 0) CHUẨN HOÁ GIÁ VỀ VND =========
# Hỗ trợ: triệu, tr, k, nghìn, ngàn, vnd, đ; USD tokens. Tạo thêm cột hiển thị triệu VND.
if has("price"):
    colp = VN["price"]
    s_raw = df[colp].astype(str)
    s_low = s_raw.str.lower().str.strip()

    # Flags theo đơn vị
    is_usd = s_low.str.contains(r"\$|usd|us\s*\$", regex=True, na=False)
    is_trieu = s_low.str.contains(r"triệu|tr\b", regex=True, na=False)
    is_nghin = s_low.str.contains(r"ngh?ìn|k\b", regex=True, na=False)

    # Lấy phần số (giữ dấu , .)
    num_txt = s_low.str.replace(r"[^0-9,\.]", "", regex=True)
    # Nếu có cả ',' và '.', cố gắng xác định: nếu số có dạng '1,234.56' → bỏ ','; nếu '1.234,56' → đổi '.'→'' và ','→'.'
    both = num_txt.str.contains(r"," , na=False) & num_txt.str.contains(r"\.", na=False)
    num_series = pd.Series(num_txt)
    num_txt = np.where(
        both,
        num_series.str.replace(r"\.(?=.*\.)", "", regex=True).str.replace(",", ".", regex=False),
        num_series
    )
    num_txt = pd.Series(num_txt).str.replace(",", ".", regex=False)
    val = pd.to_numeric(num_txt, errors="coerce")

    fx_usd_to_vnd = 25000.0

    # Quy đổi theo token đơn vị rõ ràng
    vnd = np.where(is_trieu, val * 1_000_000.0,
          np.where(is_nghin, val * 1_000.0,
            np.where(is_usd, val * fx_usd_to_vnd, val)))
    vnd = pd.to_numeric(vnd, errors="coerce")

    # Scale-up nếu có khả năng bị mất 000: không có token đơn vị, giá trị nhỏ bất thường
    unknown_unit = (~is_trieu) & (~is_nghin) & (~is_usd)
    scale1000_mask = unknown_unit & val.notna() & (val < 300_000)
    vnd = np.where(scale1000_mask, val * 1_000.0, vnd)

    # Rollback các điểm có thể USD-convert sai (ví dụ 8,999 USD → 224,975,000 VND nhưng đáng ra 8,999,000 VND)
    ratio = pd.Series(vnd) / fx_usd_to_vnd
    ratio_round = ratio.round()
    mask_integerish = ratio.notna() & (ratio - ratio_round).abs().lt(1e-6)
    mask_end = ratio_round.astype('Int64') % 100
    mis_usd = is_usd & (~is_trieu) & (~is_nghin) & mask_integerish & ratio_round.between(1000, 10000, inclusive='both') & mask_end.isin([99, 90])
    vnd = np.where(mis_usd, pd.Series(vnd) / 25.0, vnd)

    # Sửa lỗi scale nếu median bất thường
    med_vnd = pd.Series(vnd).median(skipna=True)
    if pd.notna(med_vnd) and med_vnd > 1_000_000_000:
        vnd = vnd / 1000.0

    df[colp] = pd.to_numeric(vnd, errors='coerce')
    df["Giá (triệu VND)"] = np.where(pd.notna(df[colp]), df[colp] / 1_000_000.0, np.nan)

# ========= 1) LÀM SẠCH TEXT =========
def clean_text_light(s):
    if pd.isna(s): return s
    s = str(s).strip()
    s2 = re.sub(r'[^0-9a-zA-ZÀ-ỹ\-\+\/\.\s]', '', s)
    s2 = re.sub(r'\s+', ' ', s2).strip().lower()
    return s2

for col in [VN[k] for k in ["brand","model","cpu_brand","os"] if has(k)]:
    df[col] = df[col].apply(clean_text_light)

# Ví dụ chuẩn hóa thương hiệu
if has("brand"):
    brand_map = {"samsung electronics": "samsung", "apple inc": "apple"}
    df[VN["brand"]] = df[VN["brand"]].replace(brand_map)

# ========= 2) ÉP KIỂU SỐ =========
num_cols = [VN[k] for k in ["price","rating","cpu_speed","battery","ram","rom","screen",
                            "refresh","rear_count","front_count","rear_cam","front_cam",
                            "extended_upto","res_w","res_h","cores"] if has(k)]
for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# ========= 3) CHUẨN HOÁ DỮ LIỆU PHỤC VỤ VẼ =========
# Sạc nhanh -> tách 2 trường: Có_sạc_nhanh_(0_1) và Công_suất_sạc_nhanh_(W)
if has("fast"):
    col = VN["fast"]
    s = df[col].astype(str).str.strip().str.lower()
    fast_watt = pd.to_numeric(s.str.extract(r'([0-9]+\.?[0-9]*)', expand=False), errors="coerce")
    has_fast = np.where(
        s.isin(["không hỗ trợ","khong ho tro","no","false","none","nan","", "0", "0.0"]),
        0,
        np.where(~fast_watt.isna(), 1,
                 np.where(s.isin(["yes","true","có","co","supported","co ho tro"]), 1, np.nan))
    )
    df["Có_sạc_nhanh_(0_1)"] = pd.Series(has_fast).astype("Int64")
    df["Công_suất_sạc_nhanh_(W)"] = np.where(df["Có_sạc_nhanh_(0_1)"]==1, fast_watt, 0)

# Hỗ trợ thẻ nhớ
if has("extended_upto"):
    df[VN["extended_upto"]] = df[VN["extended_upto"]].fillna(0)
    df["Có_hỗ_trợ_thẻ_nhớ_(0_1)"] = (df[VN["extended_upto"]] > 0).astype(int)

# Tính PPI
if all(has(k) for k in ["res_w","res_h","screen"]):
    df["ppi"] = np.where(
        (df[VN["screen"]]>0) & df[VN["res_w"]].notna() & df[VN["res_h"]].notna(),
        np.sqrt(df[VN["res_w"]]**2 + df[VN["res_h"]]**2) / df[VN["screen"]],
        np.nan
    )

# ========= 4) TẠO BIN / LOG =========
# Nhóm giá + log
if has("price"):
    price_m = df[VN["price"]] / 1_000_000.0
    bins = [-np.inf, 3, 5, 7, 10, 15, 20, 30, np.inf]
    labels = ["<3M","3–5M","5–7M","7–10M","10–15M","15–20M","20–30M",">30M"]
    df["Nhóm giá (triệu)"] = pd.cut(price_m, bins=bins, labels=labels, include_lowest=True, right=False)
    df["log10_giá"] = np.where(df[VN["price"]]>0, np.log10(df[VN["price"]]), np.nan)

# RAM / ROM / Màn hình
if has("ram"):
    df["Nhóm RAM"] = pd.cut(df[VN["ram"]], bins=[0,4,8,12,100],
                            labels=["≤4GB","6–8GB","12GB",">12GB"], include_lowest=True)
if has("rom"):
    df["Nhóm ROM"] = pd.cut(df[VN["rom"]], bins=[0,64,128,256,1024,10000],
                            labels=["≤64","128","256","512–1TB",">1TB"], include_lowest=True)
if has("screen"):
    df["Nhóm màn hình"] = pd.cut(df[VN["screen"]], bins=[0,6,6.7,10],
                                 labels=['<6','6–6.7','>6.7'], include_lowest=True)

# ========= 5) LƯU FILE KẾT QUẢ =========
df.to_csv('smartphones_vietnamese_viz.csv', index=False, encoding='utf-8-sig')
print("✅ Đã chuẩn hoá cho trực quan hoá và lưu file smartphones_vietnamese_viz.csv")

✅ Đã chuẩn hoá cho trực quan hoá và lưu file smartphones_vietnamese_viz.csv


In [48]:
import pandas as pd
import os

# Đọc lại bảng đã chuẩn hoá cho trực quan
df = pd.read_csv('smartphones_vietnamese_viz.csv')

# Nếu chưa có surrogate key cho fact thì tạo
if 'smartphone_sk' not in df.columns:
    df = df.reset_index(drop=True)
    # Ổn định bằng cách sort theo một số cột chính trước khi gán
    sort_cols = [c for c in ['Thương hiệu','Mẫu máy','Giá (VNĐ)'] if c in df.columns]
    if sort_cols:
        df = df.sort_values(sort_cols, kind='stable').reset_index(drop=True)
    df['smartphone_sk'] = df.index + 1

# ====== BẢNG DIMENSION ======
# Thương hiệu
if 'Thương hiệu' in df.columns:
    dim_thuong_hieu = df[['Thương hiệu']].drop_duplicates().sort_values('Thương hiệu', kind='stable').reset_index(drop=True)
    dim_thuong_hieu['ma_thuong_hieu'] = dim_thuong_hieu.index + 1
else:
    dim_thuong_hieu = pd.DataFrame(columns=['Thương hiệu','ma_thuong_hieu'])

# Bộ xử lý (CPU)
cpu_cols = [c for c in ['Hãng CPU','Số nhân CPU','Tốc độ CPU (GHz)'] if c in df.columns]
if cpu_cols:
    dim_cpu = df[cpu_cols].drop_duplicates().sort_values(cpu_cols, kind='stable').reset_index(drop=True)
    dim_cpu['ma_cpu'] = dim_cpu.index + 1
else:
    dim_cpu = pd.DataFrame(columns=['Hãng CPU','Số nhân CPU','Tốc độ CPU (GHz)','ma_cpu'])

# Màn hình
cols_disp = [c for c in ['Kích thước màn hình (inch)','Tần số quét (Hz)','Độ phân giải ngang','Độ phân giải dọc','ppi'] if c in df.columns]
if cols_disp:
    dim_man_hinh = df[cols_disp].drop_duplicates().sort_values(cols_disp, kind='stable').reset_index(drop=True)
    dim_man_hinh['ma_manhinh'] = dim_man_hinh.index + 1
else:
    dim_man_hinh = pd.DataFrame(columns=cols_disp + ['ma_manhinh'])

# Bộ nhớ
cols_storage = [c for c in ['Dung lượng RAM (GB)','Bộ nhớ trong (GB)','Hỗ trợ thẻ nhớ tối đa (GB)','Có_hỗ_trợ_thẻ_nhớ_(0_1)'] if c in df.columns]
if cols_storage:
    dim_bo_nho = df[cols_storage].drop_duplicates().sort_values(cols_storage, kind='stable').reset_index(drop=True)
    dim_bo_nho['ma_bonho'] = dim_bo_nho.index + 1
else:
    dim_bo_nho = pd.DataFrame(columns=cols_storage + ['ma_bonho'])

# Camera
cols_cam = [c for c in ['Camera sau (MP)','Camera trước (MP)','Số camera sau','Số camera trước'] if c in df.columns]
if cols_cam:
    dim_camera = df[cols_cam].drop_duplicates().sort_values(cols_cam, kind='stable').reset_index(drop=True)
    dim_camera['ma_camera'] = dim_camera.index + 1
else:
    dim_camera = pd.DataFrame(columns=cols_cam + ['ma_camera'])

# Hệ điều hành
if 'Hệ điều hành' in df.columns:
    dim_os = df[['Hệ điều hành']].drop_duplicates().sort_values('Hệ điều hành', kind='stable').reset_index(drop=True)
    dim_os['ma_os'] = dim_os.index + 1
else:
    dim_os = pd.DataFrame(columns=['Hệ điều hành','ma_os'])

# ====== BẢNG FACT ======
df_fact = df.copy()

if 'Thương hiệu' in df_fact.columns:
    df_fact = df_fact.merge(dim_thuong_hieu, on='Thương hiệu', how='left')
if cpu_cols:
    df_fact = df_fact.merge(dim_cpu, on=cpu_cols, how='left')
if cols_disp:
    df_fact = df_fact.merge(dim_man_hinh, on=cols_disp, how='left')
if cols_storage:
    df_fact = df_fact.merge(dim_bo_nho, on=cols_storage, how='left')
if cols_cam:
    df_fact = df_fact.merge(dim_camera, on=cols_cam, how='left')
if 'Hệ điều hành' in df_fact.columns:
    df_fact = df_fact.merge(dim_os, on='Hệ điều hành', how='left')

# Chọn các cột chính cho fact (sử dụng các cột sạc nhanh mới nếu có)
fact_cols = [
    'smartphone_sk',
    'ma_thuong_hieu','ma_cpu','ma_manhinh','ma_bonho','ma_camera','ma_os',
    'Giá (VNĐ)','Đánh giá','Nhóm giá (triệu)','Nhóm RAM','Nhóm ROM','Nhóm màn hình'
]
if 'Có_sạc_nhanh_(0_1)' in df_fact.columns and 'Công_suất_sạc_nhanh_(W)' in df_fact.columns:
    fact_cols[7:7] = ['Có_sạc_nhanh_(0_1)','Công_suất_sạc_nhanh_(W)']
elif 'Sạc nhanh' in df_fact.columns:
    fact_cols.insert(7, 'Sạc nhanh')

fact_cols = [c for c in fact_cols if c in df_fact.columns]
fact_smartphone = df_fact[fact_cols]

# ====== LƯU FILE ======
# Tạo folder nếu chưa tồn tại
output_folder = "output_dim_fact"
os.makedirs(output_folder, exist_ok=True)

dim_thuong_hieu.to_csv(f'{output_folder}/dim_thuong_hieu.csv', index=False, encoding='utf-8-sig')
dim_cpu.to_csv(f'{output_folder}/dim_cpu.csv', index=False, encoding='utf-8-sig')
dim_man_hinh.to_csv(f'{output_folder}/dim_man_hinh.csv', index=False, encoding='utf-8-sig')
dim_bo_nho.to_csv(f'{output_folder}/dim_bo_nho.csv', index=False, encoding='utf-8-sig')
dim_camera.to_csv(f'{output_folder}/dim_camera.csv', index=False, encoding='utf-8-sig')
dim_os.to_csv(f'{output_folder}/dim_os.csv', index=False, encoding='utf-8-sig')
fact_smartphone.to_csv(f'{output_folder}/fact_smartphone.csv', index=False, encoding='utf-8-sig')

print("✅ Đã tạo xong các bảng dimension và fact (toàn bộ tiếng Việt, SK ổn định hơn, sạc nhanh đã tách)!")

✅ Đã tạo xong các bảng dimension và fact (toàn bộ tiếng Việt, SK ổn định hơn, sạc nhanh đã tách)!
